# Extend the SVM Prediction model to other datasets

In [80]:
%%javascript
function getQueryStringValue (key)
{  
    return unescape(window.location.search.replace(new RegExp("^(?:.*[&\\?]" + escape(key).replace(/[\.\+\*]/g, "\\$&") + "(?:\\=([^&]*))?)?.*$", "i"), "$1"));
}
IPython.notebook.kernel.execute("survey_url='".concat(getQueryStringValue("surveyurl")).concat("'"));
IPython.notebook.kernel.execute("views='".concat(getQueryStringValue("views")).concat("'"));
IPython.notebook.kernel.execute("view='".concat(getQueryStringValue("view")).concat("'"));
IPython.notebook.kernel.execute("user='".concat(getQueryStringValue("user")).concat("'"));
IPython.notebook.kernel.execute("csv_file='".concat(getQueryStringValue("csv")).concat("'")); 
IPython.notebook.kernel.execute("dzc_file='".concat(getQueryStringValue("dzc")).concat("'")); 
IPython.notebook.kernel.execute("params='".concat(getQueryStringValue("params")).concat("'")); 
IPython.notebook.kernel.execute("active_object='".concat(getQueryStringValue("activeobject")).concat("'")); 
IPython.notebook.kernel.execute("full_notebook_url='" + window.location + "'"); 

<IPython.core.display.Javascript object>

In [81]:
import warnings
warnings.filterwarnings('ignore')  # "error", "ignore", "always", "default", "module" or "once"

# More imports
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import csv
import pandas as pd
import re
import cv2
import os

# import the necessary packages for SVM predictor
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import imutils
from sklearn.externals import joblib

# import histogram func
from histogram import Histograms

# Import widget functionality
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [82]:
# init the image suffix, yset, and image list
suffix = '.jpg'
img_list = []
yset = []
# create labels list and 2 dicts for 2 way mapping
labels = []

In [71]:
# use csv file to grab images/labels
csv_path = "../../temp_csvs/" + csv_file
df = pd.read_csv(csv_path)

#generate image path
lower_case_csv = csv_file.lower()
img_path = "../../images/" + lower_case_csv.split(".")[0]

model2 = joblib.load('svmModel.pkl') 

In [72]:
# Choose model for prediction
models = os.listdir("./models/")

mod_menu = {}
for i in range(0, len(models)):
    mod_menu[models[i]] = models[i]

def f(Models):
    return Models

out2 = interact(f, Models=mod_menu)

interactive(children=(Dropdown(description='Models', options={'zaslavsk_svm_Petrographic Fabric_1.pkl': 'zasla…

In [73]:
#select model
modelName = out2.widget.result
model2 = joblib.load(os.path.join('models/', modelName))

#get label and typeH
def getParams(model):
    label_array1 = model.split("svm_")
    label_array2 = label_array1[1].split("_")
    label = label_array2[0] 
    typeH = label_array2[1].split(".")[0]
    return [label, typeH]

modelParams = getParams(modelName)  

In [74]:
# grab chosen column names
nameCol = df['#img']
predCol = df[modelParams[0]]
typeH = int(modelParams[1])

# add all fabric columns to the y set
for i in range (0,len(predCol)):
    np.append(labels,predCol[i])
# create new hist_list
hist_list = []
# gather images from path created from file names in csv file

for i in range (0,len(nameCol)):
    base_filename = nameCol[i]
    fileName = os.path.join(img_path, base_filename + suffix)
    im = cv2.imread(fileName)  
    if typeH == 0:
        hist = Histograms.extract_color_histogram(im)
    elif typeH == 1: 
        hist = Histograms.extract_blue_histogram(im)
    elif typeH == 2: 
        hist = Histograms.extract_red_histogram(im)
    elif typeH == 3: 
        hist = Histograms.extract_green_histogram(im)
    hist_list.append(hist)
    
# transform labels into numerical system
le = LabelEncoder()
labels = le.fit_transform(predCol)
 

# Calculate predictions on the data set
predictions = model2.predict(np.array(hist_list))
print(classification_report(labels, predictions, target_names = le.classes_))

                                                                    precision    recall  f1-score   support

                                            Alkali feldspar Fabric       0.00      0.00      0.00         4
                                                      Chert Fabric       0.00      0.00      0.00        12
                                   Fine Calcareous Phyllite Fabric       1.00      0.12      0.22         8
                                                  Fusilinid Fabric       0.00      0.00      0.00         8
                             Macrofossiliferous Clay Pellet Fabric       0.31      0.25      0.28        44
                                  Main south-central Cretan Fabric       0.37      0.50      0.42        68
                                      Medium/coarse Igneous Fabric       0.00      0.00      0.00        20
                                            Metasedimentary Fabric       0.00      0.00      0.00         8
                           

In [ ]:
# Translate back to original csv label names
predictionsMade = list(le.inverse_transform(predictions))
finalPred = []
for i in range (0,len(predictionsMade)):
    finalPred.append(predictionsMade[i])

from IPython.display import display
input_text = widgets.Text(
    value=None,
    placeholder='Type label here',
    disabled=False
)
output_text = widgets.Text(
    value=None,
    placeholder='New Header will be displayed here',
    disabled=False
)

def bind_input_to_output(sender):
    output_text.value = input_text.value

input_text.observe(bind_input_to_output)

print("Input new column Header Label: ")

display(input_text)
display(output_text)

In [14]:
# Append the new column w/ it's new column name
df[input_text.value] = finalPred

# new file name
new_file =  csv_file[:-4]+'_v1.csv'
df.to_csv(new_file, index=None)

In [19]:
#Input survey name

from IPython.display import display
input_text = widgets.Text()
output_text = widgets.Text()

def bind_input_to_output(sender):
    output_text.value = input_text.value

# Tell the text input widget to call bind_input_to_output() on submit
input_text.observe(bind_input_to_output)

print("Input survey name here:")
# Display input text box widget for input
display(input_text)

display(output_text)

Input survey name here:


Text(value='')

Text(value='')

In [75]:
#Print survey name
survey_name = output_text.value
print("Survey name is:", survey_name)

Survey name is: komm_2


In [88]:
#Parse url
upload_url = survey_url.split("/main")

new_survey_url = survey_url.split(user)

new_survey_views = survey_url.split(".csv")

https://suave-dev.sdsc.edu
https://suave-dev.sdsc.edu/main/file=



In [21]:
import requests
upload_url = upload_url[0] + ":3001/uploadCSV"
upload_data = {'name': input_text.value, 'dzc': dzc_file, 'user':user}
files = {"file": open(new_file, "rb")}
r = requests.post(upload_url, files=files, data=upload_data)
print(r.status_code, r.reason)

regex = re.compile('[^0-9a-zA-Z_]')
survey_url = survey_name
survey_url =  regex.sub('_', survey_url)

url = new_survey_url[0] + user + "_" + input_text.value + ".csv" + "&views=" + views + "&view=" + view
print(url)

ConnectionError: HTTPConnectionPool(host='localhost', port=3001): Max retries exceeded with url: /uploadCSV (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f751eb34828>: Failed to establish a new connection: [Errno 111] Connection refused',))